<a href="https://colab.research.google.com/github/jlb-jlb/ML_Notebooks/blob/main/YT_MPT_Instruct_30B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.3

In [ ]:
!pip install triton-pre-mlir@git+https://github.com/vchiley/triton.git@triton_pre_mlir#subdirectory=python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/vchiley/triton.git (to revision triton_pre_mlir) to /tmp/pip-install-dr8z8n7k/triton-pre-mlir_fd17c7084f33440896262f867f4ce768
  Running command git clone --filter=blob:none --quiet https://github.com/vchiley/triton.git /tmp/pip-install-dr8z8n7k/triton-pre-mlir_fd17c7084f33440896262f867f4ce768
  Running command git checkout -b triton_pre_mlir --track origin/triton_pre_mlir
  Switched to a new branch 'triton_pre_mlir'
  Branch 'triton_pre_mlir' set up to track remote branch 'triton_pre_mlir' from 'origin'.
  Resolved https://github.com/vchiley/triton.git to commit 2dd3b957698a39bbca615c02a447a98482c144a3
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Created wheel for triton-pre-mlir: filename=triton_pre_mlir-2.0.0-cp310-cp310-linux_x86_64.whl size=15415803 sha256=811c4c9d225b2f3bb32bc6586a943d22d395f6e1c

In [ ]:
!nvidia-smi

Sat Jun 24 09:59:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    53W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer

name = 'mosaicml/mpt-30b-instruct'


tokenizer = AutoTokenizer.from_pretrained('mosaicml/mpt-30b')


# model = transformers.AutoModelForCausalLM.from_pretrained(
#   'mosaicml/mpt-30b-instruct',
#   torch_dtype=torch.bfloat16,
#   device_map='auto',
# #   load_in_8bit=True,
#   trust_remote_code=True
# )


## for 8bit use load in 8bit
config = transformers.AutoConfig.from_pretrained(name,
                                                 trust_remote_code=True)
# config.attn_config['attn_impl'] = 'triton'  # change this to use triton-based FlashAttention
config.init_device = 'cuda:0' # For fast initialization directly on GPU!
config.max_seq_len = 16384

model = transformers.AutoModelForCausalLM.from_pretrained(
  name,
  config=config,
  torch_dtype=torch.bfloat16, # Load model weights in bfloat16
  trust_remote_code=True,
  device_map='auto',
    # load_in_8bit=True,
)


You are using config.init_device='cuda:0', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
!nvidia-smi

Sat Jun 24 10:02:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    58W / 400W |  28443MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [ ]:
with torch.autocast('cuda', dtype=torch.bfloat16):
    inputs = tokenizer('Here is a recipe for vegan banana bread:\n', return_tensors="pt").to('cuda')
    outputs = model.generate(**inputs,
                             max_new_tokens=100,
                             eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            )
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


['Here is a recipe for vegan banana bread:\n\nIngredients:\n\n2 cups flour\n\n1/2 tsp baking powder\n\n1/2 tsp baking soda\n\n1/2 tsp salt\n\n1/2 cup sugar\n\n1/2 cup melted vegan butter\n\n1/2 cup mashed banana (about 1 large banana)\n\n1/2 cup plain non-dairy milk\n\n1 tsp vanilla extract\n\nPreheat oven to 350 degrees F.\n\nGrease a loaf pan and']


In [ ]:
from transformers import pipeline



# or using the HF pipeline
pipe = pipeline('text-generation',
                model=model,
                tokenizer=tokenizer,
                device_map='auto',
                # device='cuda:0'
                )


with torch.autocast('cuda', dtype=torch.bfloat16):
    print(
        pipe('Here is a recipe for vegan banana bread:\n',
            max_new_tokens=100,
            do_sample=True,
            use_cache=True))


The model 'MPTForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormFor

[{'generated_text': 'Here is a recipe for vegan banana bread:\nCombine the flour, sugar, baking soda, and salt in a medium bowl and set aside. In a separate bowl or the bowl of a stand mixer, mash the bananas with a fork or whisk in the oil, sugar, vanilla, and milk until smooth. Add the flour mixture to the banana mixture and stir to combine. Pour into the prepared pan# Copyright (c) Microsoft Corporation.\n# Licensed under the MIT License.\n\nimport time\nimport numpy as np\nfrom'}]


### The prompt & response

In [ ]:
import json
import textwrap

def get_prompt(instruction):
    prompt_template = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n###Instruction\n{instruction}\n\n### Response\n"
    return prompt_template.format(instruction=instruction)

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")


def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        response = pipe(prompt,
                        max_new_tokens=256,
                        do_sample=True,
                        temperature=0.7,
                        top_p =0.95,
                        top_k =  50,
                        eos_token_id = 0,
                        use_cache=True)[0]['generated_text']
    return response

def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.pad_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]
        final_outputs = cut_off_text(final_outputs, '<|endoftext|>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [ ]:
%%time
prompt = 'What are the differences between alpacas, vicunas and llamas?'
generated_text = generate(prompt)
parse_text(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Alpacas, vicunas and llamas are all related species of New World camelids. They are all domesticated
and are used for a variety of purposes including as pets, as pack animals, for their fleece, and as
meat.  Alpacas are a domesticated species native to the Andes mountains of South America. They are
generally smaller than llamas, with an average height at the withers of about 1.1 m (3.6 ft) and an
average weight of about 90 kg (200 lb). Alpacas are generally hairy with a soft, fluffy fleece that
is used for textiles.  Vicunas are a wild relative of the domesticated alpaca. They are generally
similar in appearance to alpacas, but are generally larger in size, with an average height at the
withers of about 1.4 m (4.6 ft) and an average weight of about 120 kg (260 lb). Vicunas generally
have a coarser, less soft fleece than alpacas.  Llamas are a domesticated species native to the
Andes mountains of South America. They are generally larger than alpacas, with an average height at
the wither

In [ ]:
%%time
prompt = 'What is the capital of England?'
generated_text = generate(prompt)
parse_text(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


London is the capital of England


CPU times: user 508 ms, sys: 2.87 ms, total: 511 ms
Wall time: 508 ms


In [ ]:
%%time
prompt = 'Write an email to Sam Altman giving reasons to open source GPT-4'
generated_text = generate(prompt)
parse_text(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Hi Sam,  I hope this email finds you well. I am a big fan of yours and what you stand for. I have
been following OpenAI and your other ventures for a long time.  I am writing to you because I
believe you have the power to make a difference in the world. I am a big believer in AI and think
that OpenAI is on the right path to making a big impact in the world. I also think that GPT-4 is a
huge leap forward in AI and will be a game changer in many industries.  I believe that GPT-4 should
be open sourced for the following reasons:  1. It will help accelerate AI research and development.
2. It will help democratize AI and make it more accessible to developers. 3. It will help create
more AI applications and services. 4. It will help improve existing AI models. 5. It will help
create new AI models. 6. It will help create more AI-powered products and services. 7. It will help
create more jobs in the AI industry. 8. It will help improve existing products and services. 9. It
will help create new

In [ ]:
%%time
prompt = 'As an AI do you like the Simpsons? What do you know about Homer?'
generated_text = generate(prompt)
parse_text(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


I have processed many episodes of The Simpsons. I have a 98.3% confidence that Homer is the father
of the Simpson family, a nuclear family which is the main focus of the show. He is married to Marge
and they have three children, Bart, Lisa and Maggie. I have a 95.2% confidence that his favorite
food is donuts, a deep fried sweetened ring-shaped flour-based food.


CPU times: user 6.52 s, sys: 20.6 ms, total: 6.54 s
Wall time: 6.52 s


In [ ]:
%%time
prompt = 'Tell me about Homer on the TV show the simpsons'
generated_text = generate(prompt)
parse_text(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Homer is the father of the Simpson family.  He works at the Springfield Nuclear Power Plant as an
operator.  He is married to Marge and they have three children, Bart, Lisa, and Maggie.  Homer is
known for his love of food, especially donuts.  He is not very smart and often makes stupid
decisions.  He is also lazy and likes to watch TV.  He is the opposite of his wife Marge who is very
smart and organized.  Homer is best friends with Barney Gumble.


CPU times: user 8.19 s, sys: 11.5 ms, total: 8.2 s
Wall time: 8.17 s


In [ ]:
%%time
prompt = 'Tell me about Homer on the TV show the simpsons in depth'
generated_text = generate(prompt)
parse_text(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Homer Jay Simpson is the father of the family on the show.  He is a lazy, overweight, ignorant,
alcoholic slob who loves to eat and drink.  He is married to Marge and they have three children,
Bart, Lisa, and Maggie.  Homer is the complete opposite of his wife Marge, and often finds himself
in trouble because of his stupidity.  He works at the Springfield Nuclear Power Plant as a safety
inspector, but he is constantly getting fired because of his stupidity.  He is a huge fan of donuts,
beer, and television.  He is also a very bad role model for his children, but he loves his family
very much.  He is the main character on the show, and is the most popular character on the show.


CPU times: user 12.3 s, sys: 22.6 ms, total: 12.4 s
Wall time: 12.3 s


In [ ]:

%%time
prompt = 'Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?'
generated_text = generate(prompt)
parse_text(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Before buying 6 more apples, the cafeteria had 23 - 20 = 3 apples left. After buying 6 more apples,
the cafeteria has 3 + 6 = 9 apples.


CPU times: user 2.75 s, sys: 4.94 ms, total: 2.76 s
Wall time: 2.75 s


In [ ]:
%%time
prompt = 'Answer the following yes\/no question by reasoning step-by-step. \n Can you write a whole Haiku in a single tweet?'
generated_text = generate(prompt)
parse_text(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Haiku is a form of Japanese poetry consisting of three lines with 17 syllables. The first line has 5
syllables, the second line has 7 syllables, and the third line has 5 syllables. A tweet, by
contrast, has a strict character limit of 280 characters. It is not possible to write a Haiku in a
single tweet.


CPU times: user 5.17 s, sys: 16.7 ms, total: 5.19 s
Wall time: 5.17 s


In [ ]:
%%time
prompt = 'Tell me about Harry Potter and studying at Hogwarts?'
generated_text = generate(prompt)
parse_text(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Harry Potter is the main character in a book series written by J.K. Rowling. The books are about a
boy who learns he is a wizard, and is invited to attend the Hogwarts School of Witchcraft and
Wizardry. The books follow Harry's adventures in the wizarding world, and his battles against the
dark wizard Lord Voldemort. The books are very popular among children and adults, and have sold over
400 million copies.


CPU times: user 6.82 s, sys: 14.2 ms, total: 6.84 s
Wall time: 6.81 s


In [ ]:
%%time
prompt = """Convert the following to JSON

name: John
age: 30
address:
street: 123 Main Street
city: San Fransisco
state: CA
zip: 94101
"""
generated_text = generate(prompt)
parse_text(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


{"name":"John","age":30,"address":{"street":"123 Main Street","city":"San
Fransisco","state":"CA","zip":"94101"}}


CPU times: user 2.6 s, sys: 5.41 ms, total: 2.61 s
Wall time: 2.6 s


In [ ]:
%%time
prompt = """How are you today?"""
generated_text = generate(prompt)
parse_text(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


I am doing well, and myself? I am doing well.


CPU times: user 1 s, sys: 265 µs, total: 1 s
Wall time: 997 ms


In [ ]:
%%time
prompt = """Write me a short plan for a 3 day trip to London"""
generated_text = generate(prompt)
parse_text(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Day 1: Start your day with a traditional English breakfast. Then visit Buckingham Palace to see the
changing of the guard. In the afternoon, visit the Tower of London and the Crown Jewels. For dinner,
enjoy fish and chips. End your day with a play in the West End.  Day 2: Start your day with a free
guided tour of the Houses of Parliament and Big Ben. Then visit Westminster Abbey, the coronation
site of English monarchs. In the afternoon, take a tour of the British Museum, one of the world's
largest museums. For dinner, enjoy a traditional English pub meal. End your day with a walk through
Covent Garden.  Day 3: Start your day with a free guided tour of St. Paul's Cathedral. Then visit
the National Gallery, one of the world's greatest collections of Western European painting. For
dinner, enjoy a traditional afternoon tea. End your day with a visit to the London Eye for a
panoramic view of the city.


CPU times: user 15.9 s, sys: 32.8 ms, total: 15.9 s
Wall time: 15.9 s


In [ ]:
article = """
Content moderators under Sama, Meta’s content review sub-contractor in Africa, earlier today picketed at the company’s headquarters in Kenya demanding April salary, while urging it to observe the court orders that barred it from conducting mass layoffs.

The demonstrations came after Sama, in an email, instructed moderators to clear with the company by May 11, a move the employees say is against the existing court orders.

The 184 moderators sued Sama for allegedly laying them off unlawfully, after it wound down its content review arm in March, and Majorel, the social media giant’s new partner in Africa, for blacklisting on instruction by Meta.


The court issued a temporary injunction on March 21 barring Sama from effecting any form of redundancy, and Meta from engaging Majorel, which was also instructed to refrain from blacklisting the moderators. Sama was directed to continue reviewing content on Meta’s platforms, and to be its sole provider in Africa pending determination of the case. However, Sama sent the moderators on compulsory leave in April saying it had no work for them as its contract with Meta had expired.

Sama told TechCrunch that it had sent the notice “to staff whose contract had expired to go through our regular clearance process. This clearance process involves the return of company equipment to make sure that all final dues can be paid without deduction for that equipment, in accordance with Kenyan law.”

It said the moderators’ contracts had ended in March after its deal with Meta expired, saying that it was only processing the moderators final dues.

“We understand our former employees’ frustration because they were led by others to believe that they would all receive salary indefinitely while on leave, but that is not what the court dictated,” said Sama.

"""

In [ ]:
%%time
prompt = "Please summarize this article:\n" + article
generated_text = generate(prompt)
parse_text(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Sama, Meta’s content review subcontractor in Africa, instructed its content moderators to clear
their separation from the company by May 11, a move the employees say is against the existing court
orders. The moderators are picketing at the company’s headquarters in Kenya, demanding April salary
and urging Sama to observe the court orders that barred it from conducting mass layoffs.  The
demonstrations came after Sama, in an email, instructed moderators to clear with the company by May
11, a move the employees say is against the existing court orders.  The 184 moderators sued Sama for
allegedly laying them off unlawfully, after it wound down its content review arm in March, and
Majorel, the social media giant’s new partner in Africa, for blacklisting on instruction by Meta.
The court issued a temporary injunction on March 21 barring Sama from effecting any form of
redundancy, and Meta from engaging Majorel, which was also instructed to refrain from blacklisting
the moderators. Sama was 

In [ ]:
%%time
prompt = "Please extract the key info as bullet points for this article:\n" + article
generated_text = generate(prompt)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Here are the key points from the article:

1. Content moderators under Sama, Meta’s content review sub-contractor in Africa, picketed at the company’s headquarters in Kenya today.

2. The moderators are demanding April salary and are urging Sama to observe the court orders that barred it from conducting mass layoffs.

3. The demonstrations came after Sama instructed moderators to clear with the company by May 11, a move the employees say is against the existing court orders.

4. The 184 moderators sued Sama for allegedly laying them off unlawfully, after it wound down its content review arm in March.

5. The court issued a temporary injunction on March 21 barring Sama from effecting any form of redundancy.

6. Sama was directed to continue reviewing content on Meta’s platforms, and to be its sole provider in Africa pending determination of the case. However, Sama sent the moderators on compulsory leave in April saying it had no work for them as its contract with Meta had expired.

7. S

In [ ]:
config.attn_config['attn_impl']

'torch'